# BÀI TẬP ipywidgets

## Đề Bài

Thực hiện lại phần demo trên dữ liệu InstaCart:
1. Sử dụng hàm 'interactive' thay cho hàm 'interact'.
2. Bổ sung thêm button sao cho sau khi chọn các options xong, phải click lên button mới thực hiện vẽ lại figure.

## Script

### Using

In [1]:
import squarify
import pandas as pd
import seaborn as sns
import ipywidgets as widgets
import IPython.display as disp
import matplotlib.pyplot as plt

### Contract Fields

In [2]:
CN_ORD_ID = 'order_id'
CN_PROD_ID = 'product_id'
CN_RE_ORDED = 'reordered'

In [3]:
CN_PROD_NAME = 'product_name'
CN_DEPT_ID = 'department_id'

In [4]:
CN_DEPT = 'department'

In [5]:
CN_AISLE_ID = 'aisle_id'
CN_AISLE = 'aisle'

In [6]:
cmaps = ['tab20', 'tab20b', 'viridis', 'YlOrRd', 'prism']

### Processing

In [7]:
ord_prods_pri_df = pd.read_csv('order_products__prior.csv')
prods_df = pd.read_csv('products.csv')
depts_df = pd.read_csv('departments.csv')
aisles_df = pd.read_csv('aisles.csv')

In [8]:
plot_treemap_df = pd.merge(ord_prods_pri_df, prods_df[[CN_PROD_ID, CN_DEPT_ID]], on=CN_PROD_ID, how='left')
plot_treemap_df = (plot_treemap_df.groupby(CN_DEPT_ID)[CN_PROD_ID].count().sort_values(ascending=False) / len(plot_treemap_df) * 100).to_frame().reset_index()
plot_treemap_df.columns = [CN_DEPT_ID, 'percentage']
plot_treemap_df = pd.merge(plot_treemap_df, depts_df, on=CN_DEPT_ID, how='left')

# Plot treemap
def plot_treemap(N, ColorMap):
    new_df = plot_treemap_df.head(N)
    new_df = pd.concat([new_df, pd.DataFrame({CN_DEPT: 'others', 'percentage': plot_treemap_df['percentage'][N:].sum()}, index=[0])], ignore_index=True)
    plt.figure(figsize=(20, 10))
    sns.set_style('whitegrid')
    sns.color_palette(ColorMap)
    sns.set_palette(ColorMap)
    sns.despine()
    squarify.plot(sizes=new_df['percentage'], label=new_df[CN_DEPT], alpha=.3)
    plt.title('Tỷ lệ phần trăm số lượng sản phẩm của từng departments')
    plt.axis('off')
    plt.show()

n_slider = widgets.IntSlider(value=10, min=10, max=20, step=1, description='N:')
clr_map_dropdown = widgets.Dropdown(options=cmaps, value='tab20', description='Color Map:')

widgets.interactive(plot_treemap, N=n_slider, ColorMap=clr_map_dropdown)

plot_treemap_op = widgets.Output()

def plot_treemap_on_btn_clked(b):
    with plot_treemap_op:
        disp.clear_output(True)
        plot_treemap(n_slider.value, clr_map_dropdown.value)

plot_treemap_btn = widgets.Button(description="Vẽ lại biểu đồ")
plot_treemap_btn.on_click(plot_treemap_on_btn_clked)

disp.display(n_slider, clr_map_dropdown, plot_treemap_btn, plot_treemap_op)

IntSlider(value=10, description='N:', max=20, min=10)

Dropdown(description='Color Map:', options=('tab20', 'tab20b', 'viridis', 'YlOrRd', 'prism'), value='tab20')

Button(description='Vẽ lại biểu đồ', style=ButtonStyle())

Output()

In [9]:
plot_ord_cnt_by_aisle_df = pd.merge(pd.merge(ord_prods_pri_df, prods_df, on=CN_PROD_ID, how='inner'), aisles_df, on=CN_AISLE_ID, how='inner')
dept_id = plot_ord_cnt_by_aisle_df[CN_DEPT_ID]
dept = depts_df[CN_DEPT]

# Plot order count by aisle
def plot_ord_cnt_by_aisle(dept_name):
    aisle_ord_cnt = plot_ord_cnt_by_aisle_df[dept_id == depts_df[dept == dept_name][CN_DEPT_ID].values[0]].groupby(CN_AISLE)[CN_ORD_ID].count().sort_values(ascending=False)
    sns.barplot(x=aisle_ord_cnt, y=aisle_ord_cnt.index, palette='viridis')
    plt.xlabel('Số lượng order')
    plt.ylabel('Tên aisle')
    plt.title(f'Số lượng order theo aisle trong department: {dept_name}')
    plt.show()

dept_names = depts_df[CN_DEPT].unique()
dept_name_dropdown = widgets.Dropdown(options=dept_names, value=dept_names[0], description='dep_name:')

widgets.interactive(plot_ord_cnt_by_aisle, dept_name=dept_name_dropdown)

plot_ord_cnt_by_aisle_op = widgets.Output()

def plot_ord_cnt_by_aisle_on_btn_clked(b):
    with plot_ord_cnt_by_aisle_op:
        disp.clear_output(True)
        plot_ord_cnt_by_aisle(dept_name_dropdown.value)

plot_ord_cnt_by_aisle_btn = widgets.Button(description="Vẽ lại biểu đồ")
plot_ord_cnt_by_aisle_btn.on_click(plot_ord_cnt_by_aisle_on_btn_clked)

disp.display(dept_name_dropdown, plot_ord_cnt_by_aisle_btn, plot_ord_cnt_by_aisle_op)

Dropdown(description='dep_name:', options=('frozen', 'other', 'bakery', 'produce', 'alcohol', 'international',…

Button(description='Vẽ lại biểu đồ', style=ButtonStyle())

Output()